# Дембицкий М.Е. Домашнее задание № 5

## Оглавление
[Задание 1. Установить LightFM](#task1)  
[Задание 2. Получитьи предикт, испльзую LightFM](#task2)  



<a name='task1'></a>
## Задание 1. Установить LightFM

In [ ]:
# pip install implicit

In [7]:
#pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 5.3 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-linux_x86_64.whl size=746186 sha256=08490824abf80dfaaa596c84d7296b9b2b1ea10e2c51ef16644a9fbf759ca0c0
  Stored in directory: /root/.cache/pip/wheels/ec/bb/51/9c487d021c1373b691d13cadca0b65b6852627b1f3f43550fa
Successfully built lightfm


In [15]:
# Подгружаем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Для подгрузки написанных нами ранее скриптов и csv
from google.colab import files

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k


In [14]:
# Подгружаем локальные скрипты recommenders.py, utils.py и metrics.py и csv файлы в colab репозиторий
files.upload()

Saving hh_demographic.csv to hh_demographic.csv
Saving predictions_basic.csv to predictions_basic.csv
Saving product.csv to product.csv
Saving retail_train.csv to retail_train.csv


In [47]:
# Подгружаем скрипты

from metrics import precision_at_k, recall_at_k
from utils import prefilter_items

In [19]:
# Подгружаем данные 
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [26]:
item_features.head(2)


,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [21]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [22]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [23]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [24]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

<a name='task2'></a>
## Задание 2. Получитьи предикт, испльзую LightFM

### 1. Filter items

In [28]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 2490


### 2. Prepare data set

In [29]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
# data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,26738,27030,27657,27762,30339,30374,30801,30808,31999,32457,...,10281900,10282048,10285334,10285433,10285464,10307378,10309980,10340973,10364237,10457083
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]
test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [31]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

### 3. Prepare user and item features

In [32]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
7,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown


In [33]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [34]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
7,0,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [35]:
model = LightFM(no_components=30,
                loss='bpr', # 'warp'
                learning_rate=0.05, 
                item_alpha=0.1, user_alpha=0.1, 
                random_state=42)

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4) 

In [36]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [37]:
user_emb[0]  # biases

array([-0.3575002 , -0.4004323 ,  0.        ,  0.        , -0.3036269 ,
       -0.39001995, -0.32521302,  0.        , -0.39083135, -0.30335277,
        0.        , -0.36443174, -0.35404754,  0.        ,  0.        ,
        0.        , -0.34689802,  0.        , -0.30127412,  0.        ,
       -0.298671  ,  0.        , -0.32248813, -0.27984372,  0.        ,
        0.        , -0.21886651,  0.        ,  0.        ,  0.        ,
       -0.3563947 ,  0.        ,  0.        , -0.25472796,  0.        ,
        0.        ,  0.        , -0.36276817, -0.32761532,  0.        ,
        0.        , -0.3806827 ,  0.        , -0.33369792, -0.36280996,
        0.        , -0.30405822,  0.        ,  0.        ,  0.        ,
       -0.37046438, -0.39001995,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.30171734,  0.        ,
       -0.39004815, -0.35882324,  0.        ,  0.        ,  0.        ,
       -0.29904577, -0.29327154,  0.        , -0.33146986, -0.30

In [38]:
user_emb[0].shape

(703,)

In [39]:
user_emb[1]  # embeddings

array([[ 2.2587369e-14,  6.7501174e-14, -4.4671112e-14, ...,
         2.8251456e-14, -4.1524506e-14, -2.1114397e-14],
       [ 1.7639090e-14, -1.3480382e-14, -4.8945453e-14, ...,
        -1.4651624e-15, -5.1070869e-14,  2.1893139e-14],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [40]:
user_emb[1].shape

(703, 30)

In [41]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [42]:
item_emb[0].shape

(2490,)

In [43]:
item_emb[1].shape

(2490, 30)

In [53]:
test_item_ids = np.array([1,2,3,200,1200,1,2,3,200,1200])

predictions = model.predict(user_ids=np.array([0,0,0,0,0,1,1,1,1,1]), item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)

In [57]:
predictions

array([-0.35850614, -0.35479522, -0.35850614, -0.360024  , -0.35963067,
       -0.40143824, -0.3977273 , -0.40143824, -0.4029561 , -0.40256277],
      dtype=float32)